# 文本预处理

对于序列数据处理问题，我们在 `sec_sequence` 中
评估了所需的统计工具和预测时面临的挑战。
**这样的数据存在许多种形式，文本是最常见例子之一。**
例如，一篇文章可以被简单地看作一串单词序列，甚至是一串字符序列。
本节中，我们将解析**文本的常见预处理步骤。**
这些步骤通常包括：

1. 将文本作为字符串加载到内存中。
1. 将字符串拆分为词元（如单词和字符）。
1. **建立一个词表，将拆分的词元映射到数字索引。**
1. **将文本转换为数字索引序列**，方便模型操作。

In [9]:
import collections
import re
from d2l import torch as d2l

## 读取数据集

首先，我们**从H.G.Well的[时光机器](https://www.gutenberg.org/ebooks/35)中加载文本。**
这是一个相当小的语料库，只有30000多个单词，但足够我们小试牛刀，
而现实中的文档集合可能会包含数十亿个单词。
下面的函数(**将数据集读取到由多条文本行组成的列表中**)，其中每条文本行都是一个字符串。
为简单起见，我们在这里忽略了标点符号和字母大写。


In [10]:
# 设置数据集的下载链接和校验和，存储在数据字典中
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt', '090b5e7e70c295757f55df93cb0a180b9691891a')

# 定义一个函数，用于加载和预处理时间机器数据集
def read_time_machine():
    # 使用上面定义的下载链接下载时间机器数据集，返回本地文件路径
    with open(d2l.download('time_machine'), 'r') as f:
        # 读取文件的所有行，并存储在列表中
        lines = f.readlines()
    
    # 对每一行文本进行处理：
    return [
        # 使用正则表达式替换非字母字符为空格，保留字母字符
        re.sub('[^A-Za-z]+', ' ', line)
        # 删除每行开头和结尾的空格和换行符
        .strip()
        # 将文本转换为小写字母
        .lower()
        # 遍历所有行并将结果存储在一个列表中
        for line in lines
    ]

# 调用read_time_machine函数加载和处理文本数据集，并将结果存储在变量lines中
lines = read_time_machine()

# 打印处理后的文本行的总数
print(f'# 文本总行数: {len(lines)}')
# 打印处理后的第一行文本
print(lines[0])
# 打印处理后的第11行文本
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


## 词元化

下面的`tokenize`函数**将文本行列表（`lines`）作为输入，**
**列表中的每个元素是一个文本序列（如一条文本行）。**
[**每个文本序列又被拆分成一个词元列表**]，**词元**（token）是文本的基本单位。
最后，返回一个由词元列表组成的列表，**其中的每个词元都是一个字符串（string）。**

In [11]:
# 定义一个函数，用于将文本行拆分为单词或字符词元
def tokenize(lines, token='word'):
    """
    Args:
    - lines: 包含文本行的列表，每个文本行可以是一个句子或段落
    - token: 词元类型，可以是'word'（拆分为单词）或'char'（拆分为字符）

    Returns:
    - 一个由词元列表组成的列表，其中每个词元是一个字符串
    """
    if token == 'word':
        # 如果选择词元类型为'word'，则将每个文本行按空格拆分为单词列表
        return [line.split() for line in lines]
    elif token == 'char':
        # 如果选择词元类型为'char'，则将每个文本行拆分为字符列表
        return [list(line) for line in lines]
    else:
        # 如果提供的词元类型不是'word'或'char'，则打印错误信息
        print('错误：未知词元类型：' + token)

# 使用tokenize函数将文本行列表(lines)拆分为单词词元列表(tokens)
tokens = tokenize(lines)

# 打印前11行的词元列表（每个词元列表包含单词或字符）
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


## 词表

词元的类型是字符串，而模型需要的输入是数字，因此这种类型不方便模型使用。
现在，让我们[**构建一个字典，通常也叫做*词表*（vocabulary），
用来将字符串类型的词元映射到从$0$开始的数字索引中**]。
我们先将训练集中的所有文档合并在一起，**对它们的唯一词元进行统计，**
**得到的统计结果称之为语料（corpus）。**
**然后根据每个唯一词元的出现频率，为其分配一个数字索引。**
**很少出现的词元通常被移除**，这可以降低复杂性。
另外，语料库中不存在或已删除的任何词元都将映射到一个特定的未知词元“&lt;unk&gt;”。
我们可以选择增加一个列表，用于保存那些被保留的词元，
例如：填充词元（“&lt;pad&gt;”）；
序列开始词元（“&lt;bos&gt;”）；
序列结束词元（“&lt;eos&gt;”）。


In [12]:
class Vocab:
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        # 如果没有提供tokens参数，默认为空列表
        if tokens is None:
            tokens = []
        # 如果没有提供reserved_tokens参数，默认为空列表
        if reserved_tokens is None:
            reserved_tokens = []
        # 统计tokens中词元的频率，并按频率从高到低排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True)
        # 未知词元的索引为0，添加到词汇表的起始位置
        self.idx_to_token = ['<unk>'] + reserved_tokens
        # 创建从词元到索引的映射字典
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        # 将高频词元添加到词汇表
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        # 返回词汇表中词元的总数
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            # 如果tokens不是列表或元组，则返回词元对应的索引，未知词元的索引为0
            return self.token_to_idx.get(tokens, self.unk)
        # 如果tokens是列表或元组，则返回所有词元对应的索引列表
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            # 如果indices不是列表或元组，则返回索引对应的词元
            return self.idx_to_token[indices]
        # 如果indices是列表或元组，则返回所有索引对应的词元列表
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):
        # 未知词元的索引为0，通过属性访问
        return 0

    @property
    def token_freqs(self):
        # 返回词元频率列表
        return self._token_freqs

def count_corpus(tokens):
    """统计词元的频率"""
    # 这里的tokens可以是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将2D列表展平成1D列表
        tokens = [token for line in tokens for token in line]
    # 使用collections.Counter统计词元频率
    return collections.Counter(tokens)

我们首先使用时光机器数据集作为语料库来[**构建词表**]，然后打印前几个高频词元及其索引。

In [13]:
# 使用Vocab类创建词汇表，传入tokens作为词汇表的构建数据
vocab = Vocab(tokens)

# 打印词汇表中的前10个词元及其对应的索引
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


现在，我们可以(**将每一条文本行转换成一个数字索引列表**)。


In [15]:
for i in [0, 10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])

文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


## 整合所有功能

在使用上述函数时，我们[**将所有功能打包到`load_corpus_time_machine`函数中**]，
该函数返回`corpus`（词元索引列表）和`vocab`（时光机器语料库的词表）。
我们在这里所做的改变是：

1. 为了简化后面章节中的训练，**我们使用字符（而不是单词）实现文本词元化；**
1. **时光机器数据集中的每个文本行不一定是一个句子或一个段落，还可能是一个单词，因此返回的`corpus`仅处理为单个列表，而不是使用多词元列表构成的一个列表。**


In [15]:
# 定义函数，用于加载和处理“时光机器”数据集，返回词元索引列表和词汇表对象
def load_corpus_time_machine(max_tokens=-1):
    """返回时光机器数据集的词元索引列表和词汇表"""
    
    # 1. 读取时光机器数据集中的文本行
    lines = read_time_machine()
    
    # 2. 将文本行拆分成字符词元列表
    tokens = tokenize(lines, 'char')
    
    # 3. 创建词汇表对象，用于构建词元索引和词元到索引的映射
    vocab = Vocab(tokens)
    
    # 4. 将所有词元展平成一个列表
    corpus = [vocab[token] for line in tokens for token in line]
    
    # 5. 如果提供了max_tokens参数且大于0，则限制词元索引列表的最大长度
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    
    # 6. 返回词元索引列表和词汇表对象
    return corpus, vocab

# 调用load_corpus_time_machine函数加载和处理数据集，获取词元索引列表和词汇表对象
corpus, vocab = load_corpus_time_machine()

# 获取词元索引列表corpus和词汇表vocab的长度，并打印出来
len(corpus), len(vocab)

(170580, 28)

## 小结

* 文本是序列数据的一种最常见的形式之一。
* 为了对文本进行预处理，我们通常将文本拆分为词元，构建词表将词元字符串映射为数字索引，并将文本数据转换为词元索引以供模型操作。

## 练习

1. 词元化是一个关键的预处理步骤，它因语言而异。尝试找到另外三种常用的词元化文本的方法。
1. 在本节的实验中，将文本词元为单词和更改`Vocab`实例的`min_freq`参数。这对词表大小有何影响？

[Discussions](https://discuss.d2l.ai/t/2094)
